In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [2]:
ts_df = pd.read_csv('./data/SWOW_prediction/eval/time_series/ts_df.csv')
ts_df = ts_df.groupby(['words','year','property'])[['outputs_z','count']].mean().reset_index()
ts_df = ts_df.loc[ts_df['count'] >= 50]

In [3]:
words = ts_df.words.unique()
len(words)

20995

In [4]:
words_count = ts_df.loc[ts_df.property == 'polarity']\
.groupby('words')['year'].count().to_dict()

In [5]:
words = [w for w in words_count if (words_count[w]) >= 15]
len(words)
ts_df = ts_df.loc[ts_df.words.isin(words)].reset_index(drop = True)

In [6]:
relevances = ts_df.loc[ts_df.property == 'previous_link'].\
groupby(['words','year']).outputs_z.mean().to_dict()
polarities = ts_df.loc[ts_df.property == 'polarity'].\
groupby(['words','year']).outputs_z.mean().to_dict()

In [7]:
counts = ts_df.loc[ts_df.property == 'previous_link'].\
groupby(['words','year'])['count'].mean().to_dict()

In [8]:
year_differneces_r = {year:{} for year in range(1850, 2010, 10)}
year_differneces_p = {year:{} for year in range(1850, 2010, 10)}

In [9]:
list_rows_r = []
for year in year_differneces_r:
    for word in words:
        if (word,year) not in relevances:
            continue
        
        year_relevance = np.mean([relevances[(word, y)] for y in year_differneces_r if \
                            (word,y) in relevances and \
                            y == year])
        other_relevances = [relevances[(word, y)] for y in year_differneces_r if \
                            (word,y) in relevances and \
                            y < year]
        difference = year_relevance - np.mean(other_relevances)
        year_differneces_r[year][word] = difference
        
        count1 = [counts[(word, y)] for y in year_differneces_r if \
                            (word,y) in counts and \
                            y == year]
        count1 = np.mean(count1)


        count2 = np.mean([counts[(word, y)] for y in year_differneces_r if \
                            (word,y) in counts and \
                            y < year])
        list_rows_r.append({
            'year':year,
            'word':word,
            'difference':difference,
            'n': len(other_relevances),
            'count1':count1,
            'count2':count2,
            'outputs_z_1': year_relevance,
            'outputs_z_2': np.mean(other_relevances),
            'count_diff':count1 - count2
        })

list_rows_p = []
for year in year_differneces_p:
    for word in words:
        if (word,year) not in polarities:
            continue
        year_polarity = np.mean([polarities[(word, y)] for y in year_differneces_p if \
                            (word,y) in polarities and \
                            y == year])
        other_polarities = [polarities[(word, y)] for y in year_differneces_p if \
                            (word,y) in polarities and \
                            y < year]
        difference = year_polarity - np.mean(other_polarities)
        year_differneces_p[year][word] = difference
        
        count1 = [counts[(word, y)] for y in year_differneces_r if \
                            (word,y) in counts and \
                            y == year]
        count1 = np.mean(count1)
        count2 = np.mean([counts[(word, y)] for y in year_differneces_r if \
                            (word,y) in counts and \
                            y < year])
        list_rows_p.append({
            'year':year,
            'word':word,
            'difference':difference,
            'n': len(other_polarities),
            'count1':count1,
            'count2':count2,
             'outputs_z_1': year_polarity,
            'outputs_z_2': np.mean(other_polarities),
            'count_diff':count1 - count2
        })

/home/armzn/venv2/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/armzn/venv2/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [10]:
relevance_change_df = pd.DataFrame(list_rows_r).dropna()
polarity_change_df = pd.DataFrame(list_rows_p).dropna()


In [11]:
year_relevance_mean = relevance_change_df.groupby('year').difference.mean().to_dict()
year_relevance_std = relevance_change_df.groupby('year').difference.std().to_dict()
relevance_change_df['difference_z'] = [(x - year_relevance_mean[y]) / year_relevance_std[y]\
                                       for (x, y) in zip(relevance_change_df.difference,\
                                       relevance_change_df.year)]


In [12]:
relevance_change_df = relevance_change_df.set_index(['word','year']).\
join(polarity_change_df.set_index(['word','year']),rsuffix = '_polarity').reset_index()

In [13]:
chaning_words_df = relevance_change_df.loc[(relevance_change_df.difference_z > 1.96)]

In [18]:

chaning_words_df['p'] = [1 - st.norm.cdf(x) for x in chaning_words_df['difference_z']]

In [19]:
#Now let's print the most changing words each decade
from IPython.display import display, HTML

In [20]:
years = sorted(chaning_words_df.year.unique())

In [21]:
chaning_words_df = chaning_words_df.sort_values(by = 'difference_z',ascending = False).groupby('word').\
head(1).reset_index(drop = True)

In [22]:
for year in years:
    print(year)
    display(chaning_words_df.loc[(chaning_words_df.year == year) & \
         (chaning_words_df.p < 0.01) & (chaning_words_df.count1 >= 300)]\
           .sort_values(by = 'difference_z',ascending = False).head(50))
    
    

1860


,word,year,difference,n,count1,count2,outputs_z_1,outputs_z_2,count_diff,difference_z,difference_polarity,n_polarity,count1_polarity,count2_polarity,outputs_z_1_polarity,outputs_z_2_polarity,count_diff_polarity,p
2,grey,1860,1.182878,1,761.0,289.0,0.390170,-0.792708,472.0,10.225490,0.470488,1,761.0,289.0,-0.102052,-0.572541,472.0,0.000000e+00
31,staff,1860,0.741106,1,430.0,268.0,1.146364,0.405259,162.0,6.356494,0.019183,1,430.0,268.0,0.940870,0.921687,162.0,1.032054e-10
48,hannah,1860,0.643431,1,835.0,237.0,1.291094,0.647663,598.0,5.501066,0.058141,1,835.0,237.0,0.266890,0.208749,598.0,1.887514e-08
55,witch,1860,0.632528,1,342.0,204.0,0.994257,0.361729,138.0,5.405584,0.133514,1,342.0,204.0,0.030088,-0.103425,138.0,3.229883e-08
73,martial,1860,0.575748,1,318.0,191.0,1.683624,1.107876,127.0,4.908309,-0.238689,1,318.0,191.0,-0.421894,-0.183206,127.0,4.593258e-07
111,neutral,1860,0.510058,1,310.0,151.0,1.387245,0.877187,159.0,4.333002,-0.072531,1,310.0,151.0,-0.151711,-0.079180,159.0,7.354505e-06
127,queen,1860,0.492753,1,1133.0,1781.0,0.893490,0.400737,-648.0,4.181443,-0.073927,1,1133.0,1781.0,0.376130,0.450057,-648.0,1.448327e-05
138,shoot,1860,0.481973,1,1381.0,1501.0,1.150910,0.668938,-120.0,4.087030,-0.619353,1,1381.0,1501.0,-1.675957,-1.056604,-120.0,2.184655e-05
147,column,1860,0.475330,1,1171.0,638.0,0.362630,-0.112700,533.0,4.028854,-0.012925,1,1171.0,638.0,0.117935,0.130860,533.0,2.802472e-05
149,richmond,1860,0.474786,1,581.0,124.0,1.154717,0.679931,457.0,4.024088,-0.426879,1,581.0,124.0,-2.986833,-2.559953,457.0,2.859822e-05


1870


,word,year,difference,n,count1,count2,outputs_z_1,outputs_z_2,count_diff,difference_z,difference_polarity,n_polarity,count1_polarity,count2_polarity,outputs_z_1_polarity,outputs_z_2_polarity,count_diff_polarity,p
58,plate,1870,0.581721,2,602.0,844.0,-0.159096,-0.740817,-242.0,5.311511,0.178876,2,602.0,844.0,0.088813,-0.090062,-242.0,5.436007e-08
81,lincoln,1870,0.513668,2,498.0,470.0,1.067446,0.553778,28.0,4.711151,0.229826,2,498.0,470.0,0.174999,-0.054826,28.0,1.231607e-06
97,corp,1870,0.489175,2,890.0,318.5,1.527635,1.038460,571.5,4.495077,-0.358250,2,890.0,318.5,-0.054375,0.303875,571.5,3.477232e-06
119,hood,1870,0.464513,2,410.0,134.0,-0.107814,-0.572327,276.0,4.277515,-0.166630,2,410.0,134.0,-0.308778,-0.142148,276.0,9.449557e-06
131,commander,1870,0.449524,2,496.0,654.5,2.193289,1.743765,-158.5,4.145289,-0.250698,2,496.0,654.5,0.636453,0.887150,-158.5,1.696928e-05
135,murray,1870,0.444347,2,624.0,182.5,0.638841,0.194495,441.5,4.099612,-0.343272,2,624.0,182.5,-1.985292,-1.642020,441.5,2.069214e-05
157,riot,1870,0.428297,2,338.0,168.5,1.672686,1.244389,169.5,3.958023,0.144541,2,338.0,168.5,-1.990357,-2.134897,169.5,3.778639e-05
164,tramp,1870,0.424010,2,363.0,212.5,0.048475,-0.375535,150.5,3.920203,-0.458960,2,363.0,212.5,-1.849605,-1.390645,150.5,4.423726e-05
185,roger,1870,0.409074,2,1016.0,279.5,0.084946,-0.324128,736.5,3.788440,0.012844,2,1016.0,279.5,-0.073042,-0.085886,736.5,7.579800e-05
189,mortgage,1870,0.406609,2,375.0,199.5,1.070148,0.663539,175.5,3.766694,0.001239,2,375.0,199.5,-0.105578,-0.106816,175.5,8.271169e-05


1880


...

,word,year,difference,n,count1,count2,outputs_z_1,outputs_z_2,count_diff,difference_z,difference_polarity,n_polarity,count1_polarity,count2_polarity,outputs_z_1_polarity,outputs_z_2_polarity,count_diff_polarity,p
9,bush,1980,1.414609,13,2412.0,700.230769,0.587557,-0.827051,1711.769231,8.364774,0.962490,13,2412.0,700.230769,0.251209,-0.711281,1711.769231,0.000000e+00
66,chemical,1980,0.842928,13,1364.0,573.153846,0.181696,-0.661231,790.846154,5.054542,-0.466489,13,1364.0,573.153846,-0.525735,-0.059246,790.846154,2.157123e-07
140,abuse,1980,0.671906,13,987.0,574.846154,3.971432,3.299527,412.153846,4.064265,0.091437,13,987.0,574.846154,-3.542640,-3.634077,412.153846,2.409208e-05
159,africa,1980,0.650362,13,1987.0,785.846154,0.326704,-0.323658,1201.153846,3.939522,0.209184,13,1987.0,785.846154,0.061691,-0.147493,1201.153846,4.082207e-05
216,squad,1980,0.597375,13,526.0,278.615385,2.487073,1.889697,247.384615,3.632709,-0.005123,13,526.0,278.615385,0.207422,0.212544,247.384615,1.402304e-04
251,hawk,1980,0.569850,13,1301.0,279.615385,-0.044193,-0.614043,1021.384615,3.473329,0.468191,13,1301.0,279.615385,-0.085419,-0.553610,1021.384615,2.570228e-04
311,toll,1980,0.527211,13,309.0,233.076923,0.850502,0.323291,75.923077,3.226432,-0.915408,13,309.0,233.076923,-1.971204,-1.055796,75.923077,6.267194e-04
324,group,1980,0.516644,13,9491.0,4654.230769,1.295204,0.778559,4836.769231,3.165249,-0.054760,13,9491.0,4654.230769,1.586356,1.641116,4836.769231,7.747510e-04
339,stroke,1980,0.503763,13,929.0,696.076923,0.009331,-0.494432,232.923077,3.090661,0.051968,13,929.0,696.076923,-0.192291,-0.244259,232.923077,9.985565e-04
357,african,1980,0.496876,13,1535.0,498.000000,-0.027118,-0.523994,1037.000000,3.050782,0.103195,13,1535.0,498.000000,0.570082,0.466888,1037.000000,1.141232e-03


1990


,word,year,difference,n,count1,count2,outputs_z_1,outputs_z_2,count_diff,difference_z,difference_polarity,n_polarity,count1_polarity,count2_polarity,outputs_z_1_polarity,outputs_z_2_polarity,count_diff_polarity,p
17,coach,1990,1.336326,14,2380.0,653.285714,0.921466,-0.414860,1726.714286,7.301184,0.294914,14,2380.0,653.285714,0.820774,0.525860,1726.714286,1.426637e-13
29,hazardous,1990,1.223237,14,407.0,93.357143,2.852790,1.629553,313.642857,6.706489,-3.545621,14,407.0,93.357143,-5.543726,-1.998106,313.642857,9.968137e-12
154,waste,1990,0.704789,14,2456.0,1486.000000,1.836927,1.132138,970.000000,3.980163,-1.177448,14,2456.0,1486.000000,-3.888634,-2.711187,970.000000,3.443403e-05
175,persian,1990,0.679197,14,356.0,195.714286,0.075020,-0.604177,160.285714,3.845587,0.029981,14,356.0,195.714286,0.147250,0.117270,160.285714,6.013213e-05
183,gang,1990,0.670242,14,728.0,467.357143,3.052436,2.382194,260.642857,3.798497,-0.215673,14,728.0,467.357143,-0.539062,-0.323390,260.642857,7.278811e-05
187,violence,1990,0.667104,14,1739.0,786.428571,3.305193,2.638089,952.571429,3.781991,0.319337,14,1739.0,786.428571,-3.424827,-3.744164,952.571429,7.778938e-05
261,survey,1990,0.600834,14,1721.0,864.642857,0.377940,-0.222894,856.357143,3.433502,0.138118,14,1721.0,864.642857,0.212091,0.073973,856.357143,2.979185e-04
264,gulf,1990,0.599922,14,976.0,448.142857,-0.050815,-0.650737,527.857143,3.428707,-0.034403,14,976.0,448.142857,-0.223867,-0.189464,527.857143,3.032314e-04
292,amendment,1990,0.573534,14,909.0,877.142857,1.736301,1.162767,31.857143,3.289942,0.274303,14,909.0,877.142857,1.783852,1.509549,31.857143,5.010401e-04
312,center,1990,0.560453,14,7839.0,2879.142857,0.247841,-0.312612,4959.857143,3.221156,0.205932,14,7839.0,2879.142857,1.310025,1.104093,4959.857143,6.383742e-04


2000


,word,year,difference,n,count1,count2,outputs_z_1,outputs_z_2,count_diff,difference_z,difference_polarity,n_polarity,count1_polarity,count2_polarity,outputs_z_1_polarity,outputs_z_2_polarity,count_diff_polarity,p
1,laden,2000,2.477485,15,893.0,156.000000,2.048168,-0.429317,737.000000,12.608579,-0.460068,15,893.0,156.000000,0.014761,0.474829,737.000000,0.000000e+00
10,bin,2000,1.607523,15,917.0,164.066667,1.606018,-0.001505,752.933333,8.246847,-1.365547,15,917.0,164.066667,-2.766922,-1.401376,752.933333,1.110223e-16
102,terror,2000,0.842585,15,886.0,894.466667,1.374073,0.531489,-8.466667,4.411672,0.089414,15,886.0,894.466667,-0.684755,-0.774169,-8.466667,5.128768e-06
150,dignity,2000,0.762253,15,471.0,807.666667,2.705509,1.943257,-336.666667,4.008912,0.596463,15,471.0,807.666667,2.235952,1.639489,-336.666667,3.049952e-05
153,substance,2000,0.757653,15,726.0,787.933333,1.001611,0.243958,-61.933333,3.985851,-0.499922,15,726.0,787.933333,-0.502712,-0.002790,-61.933333,3.361937e-05
162,supplement,2000,0.745502,14,467.0,264.000000,0.911914,0.166413,203.000000,3.924927,0.050394,14,467.0,264.000000,0.620193,0.569799,203.000000,4.337811e-05
207,gore,2000,0.694808,14,751.0,148.785714,2.008591,1.313783,602.214286,3.670766,1.620107,14,751.0,148.785714,0.194586,-1.425521,602.214286,1.209123e-04
209,hack,2000,0.694716,15,419.0,361.200000,0.496536,-0.198180,57.800000,3.670301,-0.181415,15,419.0,361.200000,-1.271392,-1.089976,57.800000,1.211327e-04
225,chapter,2000,0.678549,15,2134.0,2746.066667,0.520315,-0.158234,-612.066667,3.589247,-0.381579,15,2134.0,2746.066667,-0.362975,0.018604,-612.066667,1.658171e-04
255,complication,2000,0.653280,14,336.0,196.214286,0.345851,-0.307428,139.785714,3.462552,-0.394023,14,336.0,196.214286,-0.776378,-0.382354,139.785714,2.675391e-04
